# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

import os
import shutil

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperparamter-optimization'

experiment=Experiment(ws, experiment_name)

## Create a compute cluster if necessary

In [3]:
amlcompute_cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded...........................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [12]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--C': uniform(0.01, 10), '--max_iter': choice(50,100,300)})

# Setup environment for your training run
#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.', script='train.py', 
    compute_target=compute_target, environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=20, 
                                     max_concurrent_runs=4, 
                                     max_duration_minutes=10080, 
                                     policy=early_termination_policy, 
                                     run_config=estimator)



In [13]:
#TODO: Submit your experiment
hyperdrive_run=experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
hyperdrive_run.wait_for_completion(show_output= True)

RunId: HD_1e203ff6-aa35-44d8-9965-6cc1f6f60e79
Web View: https://ml.azure.com/runs/HD_1e203ff6-aa35-44d8-9965-6cc1f6f60e79?wsid=/subscriptions/eb2fe91a-2797-4c53-bd12-b644f041e7ef/resourcegroups/rg-art-ta-cfp-docker/workspaces/ml_playground&tid=c5f6f6e0-4c59-4aa1-bcd7-033f5f211b1c

Execution Summary
RunId: HD_1e203ff6-aa35-44d8-9965-6cc1f6f60e79
Web View: https://ml.azure.com/runs/HD_1e203ff6-aa35-44d8-9965-6cc1f6f60e79?wsid=/subscriptions/eb2fe91a-2797-4c53-bd12-b644f041e7ef/resourcegroups/rg-art-ta-cfp-docker/workspaces/ml_playground&tid=c5f6f6e0-4c59-4aa1-bcd7-033f5f211b1c



{'runId': 'HD_1e203ff6-aa35-44d8-9965-6cc1f6f60e79',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-05-16T07:59:35.547683Z',
 'endTimeUtc': '2023-05-16T08:17:09.277285Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8dc80376-a1cc-4f80-9905-3005a4c20bd2',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',
   'amlClientModule': '[Scrubbed]',
   'amlClientFunction': '[Scrubbed]',
   'tenantId': 'c5f6f6e0-4c59-4aa1-bcd7-033f5f211b1c',
   'amlClientRequestId': 'd720e544-3de

In [19]:
#TODO: Save the best model
best_run= hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print('Best Run Accuracy:',best_run_metrics['Accuracy'])

['--C', '6.872810472255452', '--max_iter', '50']
Best Run Accuracy: 0.8933333333333333


In [20]:
model=best_run.register_model(model_name='capstone-logistic-regression', model_path='outputs/model.joblib')
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

